In [1]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from config import *
import time

import seaborn as sns
from datetime import datetime
from pprint import pprint
from itertools import cycle

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True)

# Set target account
query = '@kaska_gorska'
# Add target account to accounts list not to analyse it
accounts = [query]
last_tweet_id = 0

In [3]:
last_tweet_id


0

In [4]:
def sentiment_bot():
    
    mentions = api.search(query, count=10, result_type='recent')
    print(len(mentions))
    
    for tweet in mentions['statuses']:
        tweet_id = tweet['id']
        print(f'tweet id: {tweet_id}')
        global last_tweet_id
        print(f'last id: {last_tweet_id}')
        
        if tweet_id > last_tweet_id:
            try:
                new_acc = (f'@{tweet["entities"]["user_mentions"][-1]["screen_name"]}')
                print(f'new acc to check: {new_acc}')
            
                if not new_acc:
                    break  
                
            except(IndexError):
                continue
            
            if new_acc not in accounts:
                accounts.append(new_acc)
                sender = (f"{tweet['user']['name']} (@{tweet['user']['screen_name']})")
                print(f'sender is {sender}')
                #print('---')
            
                blob_sent(new_acc, sender)
                #api.update_with_media('Output/plot.png', f'New Tweet Analysis of {new_acc}. Thank you {sender}!')
                api.update_with_media('Output/plot.png', f'New Tweet Analysis of .. Thank you ..!')
                print('plot printed')
            else:
                print('We\'ve analyzed it already')
                #print('---------')

    try:
        last_tweet_id = mentions['statuses'][0]['id']
        print(f'new last tweet id: {last_tweet_id}')
        
    
    except(IndexError):
        print('No new account name in the tweet')

In [5]:
def two_plots(data,send,acc):
    df = pd.DataFrame(data)
    current_date = datetime.now().date().strftime("%d.%m.%Y")
    sns.set()
    
    feature_list = ['Polarity', 'Subjectivity']
    colors = cycle(['g', '#1a75ff'])
    
    plt.figure(figsize=(7,9))
    for i in range(len(feature_list)):
        
        plt.subplot(2,1,i+1)
        plt.plot(df['Tweets Ago'], df[feature_list[i]], marker='o', linewidth=0.3, color=next(colors), alpha=0.9)
        plt.title(f'Sentiment Analysis of {acc} Tweets {feature_list[i]}\n on {current_date}. Requested by {send}')
        plt.ylabel(f'Tweets {feature_list[i]}')
        plt.ylim(-1.1,1.1)
        
    plt.tight_layout()    
    plt.savefig('Output/plot.png')

In [6]:
def blob_sent(acc,send):
    
    total_mood = []
    last_tweet = None
    tweet_counter = 0
    send = send
    acc = acc
    
    for x in range(3):
        
        all_data = api.user_timeline(acc, count=10, max_id=last_tweet, page=x)
        
        for tweet in all_data:
            
            blob = TextBlob(tweet['text'])
            sentiment = blob.sentiment
            total_mood.append({'source': acc,
                             'Polarity': sentiment[0],
                             'Subjectivity': sentiment[1],
                             'Tweets Ago': tweet_counter,
                             'text': tweet['text'],
                             'time': tweet['created_at'],
                              'likes': tweet['favorite_count'],
                              'RT': tweet['retweet_count']})
            tweet_counter -= 1
        last_tweet = tweet["id"] - 1
    
    return two_plots(total_mood,send,acc)

In [ ]:

counter = 1

while(True):
    
    sentiment_bot()
    time.sleep(40)
    counter +=1

2
No new account name in the tweet
2
No new account name in the tweet
2
No new account name in the tweet
2
No new account name in the tweet
2
No new account name in the tweet
2
No new account name in the tweet
2
No new account name in the tweet


In [ ]:

api.update_status('@kaska_gorska analyze @rock')

In [ ]:
for tweet in mentions['statuses']:
    tweet_id = tweet['id']
    print(tweet_id)
    if tweet_id > last_tweet_id:
        try:
            new_acc = (f'@{tweet["entities"]["user_mentions"][-1]["screen_name"]}')
            print(f'new acc to check: {new_acc}')
            
            if not new_acc:
                break  
                
        except(IndexError):
            print('')
            
        if new_acc not in accounts:
            accounts.append(new_acc)
            sender = (f"{tweet['user']['name']} (@{tweet['user']['screen_name']})")
             
            print(f'sender is {sender}')
            print('---')
            
            blob_sent(new_acc, sender)
            api.update_with_media('Output/plot.png', f'New Tweet Analysis of {new_acc}. Thank you {sender}!')
            
        else:
            print('We\'ve analyzed it already')
            print('---------')

try:
    mentions['statuses'][0]['id'] = tweet_id
except(IndexError):
    print('No new account name in the tweet')